In [16]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf

from transformers import SpeechEncoderDecoderModel, Speech2Text2Processor


In [ ]:


model = Speech2TextForConditionalGeneration.from_pretrained(
    "facebook/s2t-medium-librispeech-asr"
)
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-medium-librispeech-asr")



In [17]:
model=Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53")


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [18]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

In [2]:
processor = Speech2Text2Processor.from_pretrained('facebook/s2t-wav2vec2-large-en-de')
model = SpeechEncoderDecoderModel.from_pretrained('facebook/s2t-wav2vec2-large-en-de')

Downloading:   0%|          | 0.00/5.82k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/164k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [4]:
model

SpeechEncoderDecoderModel(
  (encoder): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel

In [11]:

def from_file(file):
    speech, _ = sf.read(file)
    return speech


# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
# ds = ds.map(map_to_array)

# input_features = processor(
#     ds["speech"][0], sampling_rate=16_000, return_tensors="pt"
# ).input_features  # Batch size 1

filepath = "sample/84-121123-0001.wav"
input_features = processor(
    from_file(filepath), sampling_rate=16_000, return_tensors="pt"
).input_values  # Batch size 1




In [10]:
input_features

{'input_values': tensor([[-0.0203, -0.0199, -0.0121,  ...,  0.0286,  0.0325,  0.0246]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [12]:
# https://huggingface.co/transformers/main_classes/model.html
generated_ids = model.generate(
    input_ids=input_features, num_beams=10, num_return_sequences=10
)

/root/anaconda3/envs/pyctc/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [13]:
print(generated_ids)
transcription = processor.batch_decode(generated_ids)

for t in transcription:
    print(t)

tensor([[   2,  424,    8, 1173,   29,  596, 3722,   83, 4269, 2204,    9, 5391,
          200,   41, 3661, 8529, 7970,    4,    2,    1,    1],
        [   2,  424,    8, 1173,   29,  596, 3722, 3026, 1489,   61,    9, 5391,
          200,   41, 3661, 8529, 7970,    4,    2,    1,    1],
        [   2,  424,    8, 1173,   29,  596, 3722,   83, 4269, 2204,    9, 5391,
          200, 3661, 6577, 6079, 7970,    4,    2,    1,    1],
        [   2,  424,    8, 1173,   29,  596, 3722, 3026,  170,  179,   61,    9,
         5391,  200,   41, 3661, 8529, 7970,    4,    2,    1],
        [   2,  424,    8, 1173,   29,  596, 3722,   83, 4269, 2204,    9, 5391,
          200,   41, 3661, 6577,  315, 7970,    4,    2,    1],
        [   2,  424,    8, 1173,   29,  596, 3722,   83, 4269, 2204,   22, 1706,
         5549, 1554,  200,   41, 3661, 8529, 7970,    4,    2],
        [   2,  424,    8, 1173,   29,  596, 3722, 3026, 1489,   61,    9, 5391,
          200, 3661, 6577, 6079, 7970,    4,    2

In [14]:
model.encoder

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureExtractor(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (1): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (2): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (3): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (4): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elem